In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
DATA_FILEPATH = Path.cwd() / '..' / 'data.csv'
data = pd.read_csv(DATA_FILEPATH)

In [ ]:
# Verify "ativos" is 'confirmados' minus 'recuperados' minus 'obitos'
df = data.copy()

# column 'calc' becomes 'confirmados' minus 'obitos' minus 'recuperados'
data['calc'] = data['confirmados'] - data['recuperados'] - data['obitos']
# NaN means we don't know the number. Let's make those '0' to simplify here
data.fillna(0, inplace=True)

# column 'diff' becomes the difference betwen calc and ativos
data['diff'] = data['calc'] - data['ativos']

# filter by values where diff != 0
fail = data[data['diff'] != 0]
fail = fail[['data', 'diff', 'calc', 'ativos', 'confirmados', 'obitos', 'recuperados']]
if len(fail):
    print(fail)

In [ ]:
# Verify "ativos" = "confirmados" minus "recuperados" minus "obitos" as well as
# "ativos_novos" = "confirmados_novos" minus "recuperados_novos" minus "obitos_novos"
df = data.copy()
# unknowns are zero
df.fillna(0, inplace=True)

# focus on the four values
df = df.loc[:, ['data', 'confirmados', 'recuperados', 'obitos', 'ativos']]
# add four "*_novos"
df['confirmados_novos'] = df['confirmados'].diff(1)
df['recuperados_novos'] = df['recuperados'].diff(1)
df['obitos_novos'] = df['obitos'].diff(1)
df['ativos_novos'] = df['ativos'].diff(1)
# remove first row (NaN because there's no previous day)
df = df[1:]

df['ativos_calc'] = df['confirmados'] - df['recuperados'] - df['obitos']
df['ativos_novos_calc'] = df['confirmados_novos'] - df['recuperados_novos'] - df['obitos_novos']
df['diff'] = df['ativos_calc'] - df['ativos'] + df['ativos_novos_calc'] - df['ativos_novos']

# filter by values where diff != 0
fail = df[df['diff'] != 0]
fail = fail[['data', 'diff', 'ativos_calc', 'ativos', 'ativos_novos_calc', 'ativos_novos']]
if len(fail):
    print(fail)

df['confirmados_calc'] = df['recuperados'] + df['obitos'] + df['ativos']
df['confirmados_novos_calc'] = df['recuperados_novos'] + df['obitos_novos'] + df['ativos_novos']
df['diff'] = df['confirmados_calc'] - df['confirmados'] + df['confirmados_novos_calc'] - df['confirmados_novos']

# filter by values where diff != 0
fail = df[df['diff'] != 0]
fail = fail[['data', 'diff', 'confirmados_calc', 'confirmados', 'confirmados_novos_calc', 'confirmados_novos']]
if len(fail):
    print(fail)


In [ ]:
# Verify "confirmados_novos" is "confirmados" minus previous day
df = data.copy()

# column 'calc' becomes the difference between 'confirmados' from the current row and the previous row
df['calc'] = df['confirmados'].diff(1)
# first row will be NaN because there's no previous row
df.loc[[0], ['calc']] = 0

# column 'diff' becomes the difference betwen calc and confirmados_novos
df['diff'] = df['calc'] - df['confirmados_novos']

# filter by values where diff != 0
fail = df[df['diff'] != 0]
fail = fail[['data', 'diff', 'calc', 'confirmados_novos', 'confirmados']]
if len(fail):
    print(fail)


In [ ]:
# Verify "confirmados" is the sum of all regions Portugal, including "estrangeiros"
df = data.copy()
# allow "estrangeiro" to be unknown
df.confirmados_estrangeiro.fillna(0, inplace=True)

df['calc'] = df['confirmados_arsnorte'] + df['confirmados_arscentro'] + df['confirmados_arslvt'] + df['confirmados_arsalentejo'] + df['confirmados_arsalgarve'] + df['confirmados_acores'] + df['confirmados_madeira'] + df['confirmados_estrangeiro']

df['diff'] = df['calc'] - df['confirmados']

fail = df[df['diff'] != 0]
fail = fail[['data', 'diff', 'calc', 'confirmados', 'confirmados_arsnorte', 'confirmados_arscentro', 'confirmados_arslvt', 'confirmados_arsalentejo', 'confirmados_arsalgarve', 'confirmados_acores', 'confirmados_madeira', 'confirmados_estrangeiro']]
if len(fail):
    # 16-03-2020 (-1) and #20-03-2020 (-3)
    print(fail)


In [ ]:
# Verify "confirmados" is the sum of "f" and "m"
df = data.copy()

df['calc'] = df['confirmados_f'] + df['confirmados_m']

df['diff'] = df['calc'] - df['confirmados']

fail = df[df['diff'] != 0]
# ignore initial rows where _f and _m where unknown
fail = fail[ df.confirmados_f.notnull() & df.confirmados_m.notnull() ]
fail = fail[['data', 'diff', 'calc', 'confirmados', 'confirmados_f', 'confirmados_m']]
if len(fail):
    # 27-03-2020 (-4), 03-05-2020 (-81), and 27-08-2020 (+2)
    print(fail)


In [ ]:
# Verify "confirmados" is the sum of all ages
# TODO the difference is the unknowns, which are not being populated?
df = data.copy()
# allow "desconhecidos" to be unknown
df.confirmados_desconhecidos_f.fillna(0, inplace=True)
df.confirmados_desconhecidos_m.fillna(0, inplace=True)

df['calc'] = df['confirmados_0_9_f'] + df['confirmados_0_9_m'] + df['confirmados_10_19_f'] + df['confirmados_10_19_m'] + df['confirmados_20_29_f'] + df['confirmados_20_29_m'] + df['confirmados_30_39_f'] + df['confirmados_30_39_m'] + df['confirmados_40_49_f'] + df['confirmados_40_49_m'] + df['confirmados_50_59_f'] + df['confirmados_50_59_m'] + df['confirmados_60_69_f'] + df['confirmados_60_69_m'] + df['confirmados_70_79_f'] + df['confirmados_70_79_m'] + df['confirmados_80_plus_f'] + df['confirmados_80_plus_m'] + df['confirmados_desconhecidos_m'] + df['confirmados_desconhecidos_f']

df['diff'] = df['calc'] - df['confirmados']

fail = df[df['diff'] != 0]
fail = fail[ df.confirmados_f.notnull() & df.confirmados_m.notnull() ]
fail = fail[['data', 'diff', 'calc', 'confirmados', 'confirmados_0_9_f', 'confirmados_0_9_m', 'confirmados_10_19_f', 'confirmados_10_19_m', 'confirmados_20_29_f', 'confirmados_20_29_m', 'confirmados_30_39_f', 'confirmados_30_39_m', 'confirmados_40_49_f', 'confirmados_40_49_m', 'confirmados_50_59_f', 'confirmados_50_59_m', 'confirmados_60_69_f', 'confirmados_60_69_m', 'confirmados_70_79_f', 'confirmados_70_79_m', 'confirmados_80_plus_f', 'confirmados_80_plus_m', 'confirmados_desconhecidos_m', 'confirmados_desconhecidos_f']]
#if len(fail):
#    print(fail)
fail
